In [21]:
import subprocess
import shlex
import numpy as np

In [22]:
cmd_split = shlex.split('hadoop fs -du -h /user/imoise/TwitterData/2014_05/20/10/')
p = subprocess.Popen(cmd_split, stdout=subprocess.PIPE)
outp = p.communicate()[0]

In [36]:
clean_list = [f for f in outp.split() if len(f) > 5 and f[-5:] == '.json']

In [38]:
len(clean_list)

60

In [39]:
!hadoop fs -du -h /user/imoise/TwitterData/

Picked up _JAVA_OPTIONS: -Xmx1024m -Xms256m -XX:ParallelGCThreads=1
16/05/03 17:47:42 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
525.2 G  /user/imoise/TwitterData/2012
948.4 G  /user/imoise/TwitterData/2013
311.3 G  /user/imoise/TwitterData/2014_02
428.3 G  /user/imoise/TwitterData/2014_03
420.8 G  /user/imoise/TwitterData/2014_04
415.0 G  /user/imoise/TwitterData/2014_05
242.3 G  /user/imoise/TwitterData/2014_06
390.1 G  /user/imoise/TwitterData/2014_07
398.5 G  /user/imoise/TwitterData/2014_09
467.0 G  /user/imoise/TwitterData/2014_10
472.7 G  /user/imoise/TwitterData/2014_11
492.7 G  /user/imoise/TwitterData/2014_12


In [40]:
months = ['05', '06', '07', '09', '10', '11', '12']

In [41]:
days_hours = np.zeros((len(months)*31, 24))

NameError: name 'np' is not defined

In [42]:
cmd_split = shlex.split('hadoop fs -du -h /user/imoise/TwitterData/2014_01/20/10/')
p = subprocess.Popen(cmd_split, stdout=subprocess.PIPE)
outp = p.communicate()[0]

In [45]:
clean_list = [f for f in outp.split() if len(f) > 5 and f[-5:] == '.json']

In [46]:
len(clean_list)

0

In [49]:
!hadoop fs -du -h /user/imoise/TwitterData/2014_05

Picked up _JAVA_OPTIONS: -Xmx1024m -Xms256m -XX:ParallelGCThreads=1
16/05/03 17:52:07 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
13.7 G  /user/imoise/TwitterData/2014_05/01
13.3 G  /user/imoise/TwitterData/2014_05/02
13.5 G  /user/imoise/TwitterData/2014_05/03
14.3 G  /user/imoise/TwitterData/2014_05/04
12.9 G  /user/imoise/TwitterData/2014_05/05
8.2 G   /user/imoise/TwitterData/2014_05/06
13.9 G  /user/imoise/TwitterData/2014_05/07
14.0 G  /user/imoise/TwitterData/2014_05/08
13.0 G  /user/imoise/TwitterData/2014_05/09
13.9 G  /user/imoise/TwitterData/2014_05/10
14.1 G  /user/imoise/TwitterData/2014_05/11
13.9 G  /user/imoise/TwitterData/2014_05/12
13.3 G  /user/imoise/TwitterData/2014_05/13
13.2 G  /user/imoise/TwitterData/2014_05/14
13.9 G  /user/imoise/TwitterData/2014_05/15
13.5 G  /user/imoise/TwitterData/2014_05/16
14.0 G  /user/imoise/TwitterData/2014_05/17
13.9 G  /user/imoise/TwitterData/201

In [ ]:
!hadoop fs -du -h /user/imoise/TwitterData/2014_05/